## Load libraries

In [1]:
from itertools import product
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from scipy.stats import norm
import time

## Define key parameters

In [2]:
num_items = 1000
num_dimensions = 5

items = ['item_' + str(x) for x in range(num_items)]
dimensions = ['dim_' + str(x) for x in range(num_dimensions)]

## Generate (random) input data

In [3]:
item_value = {item: norm(loc=10, scale=1).rvs() for item in items}

item_space_df = pd.DataFrame(list(product(*[items, dimensions])), columns=['item', 'dimension'])
item_space_df['space'] = np.round(norm(loc=10, scale=1).rvs(size=num_items*num_dimensions), 2)
item_space = item_space_df.set_index(['item', 'dimension']).to_dict()['space']

In [4]:
knapsack_space = {dimension: norm(loc=5*num_items, scale=0.5*num_items).rvs() for dimension in dimensions}

## Define model via pyomo

In [5]:
def define_objective(model):
    return sum(model.a[item] * model.x[item] for item in model.Items)

def define_dimension_constraint(model, dimension):
    return sum(model.b[item, dimension] * model.x[item] for item in model.Items) <= model.c[dimension]

In [6]:
model = pyo.AbstractModel()

In [7]:
model.Items = pyo.Set(initialize=items)
model.Dimensions = pyo.Set(initialize=dimensions)

In [8]:
model.a = pyo.Param(model.Items, initialize=item_value)
model.b = pyo.Param(model.Items, model.Dimensions, initialize=item_space)
model.c = pyo.Param(model.Dimensions, initialize=knapsack_space)

In [9]:
model.x = pyo.Var(model.Items, within=pyo.Binary)

In [10]:
model.obj = pyo.Objective(rule=define_objective, sense=pyo.maximize)

In [11]:
model.dimension_constraint = pyo.Constraint(model.Dimensions, rule=define_dimension_constraint)

In [12]:
start_time = time.time()

instance = model.create_instance()

print('Instance creation: ' + str(time.time() - start_time) + ' seconds')

Instance creation: 0.12227797508239746 seconds


In [13]:
opt = pyo.SolverFactory('glpk', executable='/Applications/glpk-5.0/examples/glpsol')

## Solve the model

In [14]:
solver_start_time = time.time()

results = opt.solve(instance, tee=False, keepfiles=False)

print('Solver time: ' + str(time.time() - solver_start_time) + ' seconds')

Solver time: 0.6726779937744141 seconds


In [15]:
results.solver.termination_condition

<TerminationCondition.optimal: 'optimal'>

## Inspect the results

In [16]:
for item in instance.Items:
    if instance.x[item].value > 0.5:
        print('Take item ' + item + ' of value ' + str(round(item_value[item],1)))

Take item item_1 of value 9.2
Take item item_2 of value 10.7
Take item item_4 of value 10.0
Take item item_5 of value 10.9
Take item item_6 of value 10.7
Take item item_7 of value 12.6
Take item item_9 of value 10.5
Take item item_10 of value 11.0
Take item item_11 of value 10.9
Take item item_15 of value 9.5
Take item item_17 of value 10.8
Take item item_20 of value 9.9
Take item item_24 of value 10.9
Take item item_25 of value 11.4
Take item item_27 of value 10.7
Take item item_29 of value 9.8
Take item item_30 of value 9.2
Take item item_31 of value 11.9
Take item item_34 of value 10.3
Take item item_35 of value 11.5
Take item item_38 of value 11.4
Take item item_40 of value 9.9
Take item item_41 of value 10.7
Take item item_43 of value 11.3
Take item item_44 of value 9.6
Take item item_45 of value 11.2
Take item item_47 of value 10.0
Take item item_48 of value 10.7
Take item item_50 of value 12.1
Take item item_52 of value 9.6
Take item item_53 of value 10.7
Take item item_55 of va